<a href="https://colab.research.google.com/github/AmanPriyanshu/FrequentPatternMining/blob/main/FPTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/AmanPriyanshu/FrequentPatternMining.git

Cloning into 'FrequentPatternMining'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 32 (delta 7), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (32/32), done.


In [2]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import pickle

In [3]:
with open('./FrequentPatternMining/data/user_compressed_lists.pkl', 'rb') as f:
  user_compressed_lists_loaded = pickle.load(f)

In [4]:
user_compressed_lists = [[str(j) for j in i['indexes']] for i in user_compressed_lists_loaded]

In [5]:
class Node:
  def __init__(self, value):
    self.value = value
    self.value_count = 1
    self.next_array = {}

  def next(self, value):
    self.next_array.update({value: Node(value)})

In [6]:
class Growth:
  def __init__(self, data, min_support=30):
    self.og_data = data
    self.min_support = min_support
    self.supported_item_list = self.generate_supported_item_list()
    self.data = self.generate_supported_data()
    self.root = self.generate_tree()
    self.generated_paths = self.item_wise()
    self.associations = self.generate_associations()

  def generate_supported_item_list(self):
    supported_items = {}
    for user in tqdm(self.og_data, desc="Generating supported list"):
      for item in user:
        if item not in list(supported_items.keys()):
          supported_items.update({item: 1})
        else:
          supported_items[item] += 1
    frequency = np.array([[int(i) for i in list(supported_items.keys())], [int(supported_items[i]) for i in list(supported_items.keys())]]).T
    indexes = np.argsort(frequency.T[1])[::-1]
    frequency = frequency[indexes]
    supported_item_list = [str(row[0]) for row in frequency if row[1]>=self.min_support]
    return supported_item_list

  def generate_supported_data(self):
    data = self.og_data[:]
    sorted_data = []
    for user in tqdm(data, desc="Generating Ordered Lists"):
      sorted_data.append([j for j in self.supported_item_list if j in user])
    return sorted_data

  def generate_tree(self):
    root = Node("-1")
    for user in tqdm(self.data, desc="Generating Frequency Pattern Tree"):
      curr = root
      for item in user:
        if item in curr.next_array.keys():
          curr.value_count += 1
          curr = curr.next_array[item]
        else:
          curr.next(item)
    return root

  def traverse_tree(self, node, value):
    if node.value==value:
      return True, [[node.value, node.value_count]]
    else:
      generated_paths = []
      if len(node.next_array) != 0:
        for key, next_node in node.next_array.items():
          condition, path = self.traverse_tree(next_node, value)
          if condition:
            generated_paths.extend([[node.value+"-->"+i[0], i[1]] for i in path])
        if len(generated_paths)==0:
          return False, ["None"]
        else:
          return True, generated_paths
      else:
        return False, ["None"]

  def item_wise(self):
    paths = {}
    for item in tqdm(self.supported_item_list, desc="Generating Item Pathways"):
      condition, path = self.traverse_tree(self.root, item)
      paths.update({item: path})
    return paths

  def generate_associations(self):
    generated_paths = self.generated_paths
    tmps = {}
    for key, path_list in tqdm(generated_paths.items(), desc="Generating Associations"):
      path_list = [[i[0].split('-->'), i[1]] for i in path_list]
      length_array = [len(i[0]) for i in path_list]
      max_length = max(length_array)
      longest_path = path_list[length_array.index(max_length)][0]
      for length in range(1, max_length+1):
        tmp = longest_path[:length]
        for path, value_count in path_list:
          validation = True
          for i,j in zip(tmp, path):
            if i!=j:
              validation = False
              break
          if not validation:
            tmp = tmp[:-1]
            break
      tmps.update({key:tmp})
    return tmps

  def display_associations(self):
    df = pd.read_csv("./FrequentPatternMining/data/ui_matrix.csv")
    movie_names = list(df.columns)
    associations = {key:'-->'.join([movie_names[int(j)] for j in g.associations[key] if j!=key])+'-->'+movie_names[int(key)] for key in self.associations.keys()}
    _ = [print(i) for k,i in associations.items()]

In [7]:
g = Growth(user_compressed_lists)

Generating supported list:   0%|          | 0/943 [00:00<?, ?it/s]

Generating Ordered Lists:   0%|          | 0/943 [00:00<?, ?it/s]

Generating Frequency Pattern Tree:   0%|          | 0/943 [00:00<?, ?it/s]

Generating Item Pathways:   0%|          | 0/806 [00:00<?, ?it/s]

Generating Associations:   0%|          | 0/806 [00:00<?, ?it/s]

In [8]:
g.display_associations()

Scream of Stone (Schrei aus Stein) (1991)-->I.Q. (1994)
Scream of Stone (Schrei aus Stein) (1991)-->I.Q. (1994)-->Men in Black (1997)
Scream of Stone (Schrei aus Stein) (1991)-->I.Q. (1994)-->Men in Black (1997)-->Snow White and the Seven Dwarfs (1937)
Scream of Stone (Schrei aus Stein) (1991)-->I.Q. (1994)-->Men in Black (1997)-->Snow White and the Seven Dwarfs (1937)-->Apocalypse Now (1979)
Scream of Stone (Schrei aus Stein) (1991)-->I.Q. (1994)-->Men in Black (1997)-->Snow White and the Seven Dwarfs (1937)-->Apocalypse Now (1979)-->Donnie Brasco (1997)
Scream of Stone (Schrei aus Stein) (1991)-->I.Q. (1994)-->Men in Black (1997)-->Snow White and the Seven Dwarfs (1937)-->Apocalypse Now (1979)-->Donnie Brasco (1997)-->Secrets & Lies (1996)
Scream of Stone (Schrei aus Stein) (1991)-->I.Q. (1994)-->Men in Black (1997)-->Snow White and the Seven Dwarfs (1937)-->Apocalypse Now (1979)-->Donnie Brasco (1997)-->Secrets & Lies (1996)-->Marvin's Room (1996)
Scream of Stone (Schrei aus Stein) 